# Create an emtpty MIKE+ sqlite database

This example shows to create an empty MIKE+ sqlite database. Basic workflow is:

1. Specify the coordinate system.
2. Create the empty database.
3. Configure the model type.

#### Required imports

In [ ]:
import clr
from System import Nullable

import mikeplus
from mikeplus import DataTableAccess
from DHI.Amelia.DataModule.Services.DataSource import BaseDataSource

#### 1. Specify the coordinate system

In [ ]:
# A MIKE+ compatible projection string (e.g. readable WKT of the coordinate system from MIKE+). Refer to the MIKE+ documentation for more information.
projection_string =  (
"""
PROJCS["ETRS89 / UTM zone 32N",
    GEOGCS["ETRS89",
        DATUM["European_Terrestrial_Reference_System_1989",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6258"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4258"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",9],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","25832"]]
"""
)

In [ ]:
# Alternatively, define CRS using an SRID (e.g. EPSG).
srid = 25832

In [ ]:
# If the projection string is known, then set SRID to -1. Otherwise, if using SRID, set projection_string to an empty string.
if projection_string:
    srid = -1
else:
    projection_string = ""

#### 2. Create the database

In [ ]:
# Path to model database
database_path = "my_model.sqlite"

In [ ]:
# Create a new database
data_source = BaseDataSource.Create(database_path)
data_source.CreateDatabase();


In [ ]:
# Set the coordinate system
data_source.OpenDatabase()
data_source.CreateModelTables(srid, projection_string)
data_source.CloseDatabase();

#### 3. Configure the model type.

In [ ]:
# Create a DataTableAccess object to access the database
dta = DataTableAccess(database_path)
dta.open_database()
dta

In [ ]:
# Set the active model

# CS_MIKE1D = River, collection systems and overland flow
# CS_SWMM   = SWMM collection systems and overland flow
# WD_EPANET = Water distribution

dta.datatables.SetActiveModel(dta.datatables.ActiveModel.CS_MIKE1D)
dta.datatables.ActiveModel

In [ ]:
# Get the default model type settings as a dictionary
fields = [c.Field for c in dta.datatables["m_ModelSetting"].Columns]
values = dta.get_field_values("m_ModelSetting", "MuModel", fields)
settings = {field:value for field, value in zip(fields, values)}
settings

In [ ]:
# Disable all features and modules, then enable 2D Overland and Hydrodynamic.
settings_to_update = {}
for key, value in settings.items():
    if "Enable" in key:
        settings_to_update[key] = Nullable[int](0)
settings_to_update["Enable_2DOverland"] = Nullable[int](1)
settings_to_update["Enable_HD"] = Nullable[int](1)

In [ ]:
# Update the sqlite database with the new settings
dta.set_values("m_ModelSetting", "MuModel", settings_to_update)

In [ ]:
# Close the database
dta.close_database()

#### Clean up

In [ ]:
from pathlib import Path
db_sqlite = Path(database_path)
db_mupp = db_sqlite.with_suffix(".mupp")

for file in [db_sqlite, db_mupp]:
    if file.exists():
        file.unlink()